In [1]:
import json

from pymongo import MongoClient

import numpy as np
import matplotlib.pyplot as plt
import line_profiler

from helpers import *
from imputation import *

from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import *

from ipywidgets.widgets import Button, HBox
from IPython.display import display

client = MongoClient("localhost", 27017)
db = client['usgs']
output_notebook()

Loading BokehJS ...

In [ ]:
sid = json.load(open('revision_list_d.json'))['to_review']

In [ ]:
si = 0
print(sid[si])

Tm, Zm, Tc, Zc = get_data(db, sid[si])
Zm = feet_to_meters(Zm)
Zc = feet_to_meters(Zc)
dt = min((Tc[1:] - Tc[:-1]).min(), (Tm[1:] - Tm[:-1]).min())

Ym, Yc = align_measurements(Tm, Zm, Tc, Zc)
Ym[Ym <= 0] = -1
Yc[Yc <= 0] = -1
dY = np.hstack([0, Ym[1:] - Ym[:-1]]) / (dt / 3600)
y_true = mark_anomaly(Ym, Yc, 0.02)

n = 1000
k = 50
i = k
count = 0
idx = list()   

source = ColumnDataSource(
    data = dict(
        x  = np.arange(i-k,i+n+k), 
        m  = Ym[i-k:i+n+k], 
        c  = Yc[i-k:i+n+k], 
        dy = dY[i-k:i+n+k],
        a  = y_true[i-k:i+n+k]
    )
)

gh_tools      = [BoxZoomTool(), WheelZoomTool(dimensions=['width']), WheelZoomTool(dimensions=['height']), 
                 PanTool(), SaveTool(), BoxSelectTool(dimensions=['width'])]
anomaly_tools = [BoxSelectTool(dimensions=['width']), BoxZoomTool(dimensions=['width']), 
                 WheelZoomTool(dimensions=['width']), PanTool(dimensions=['width'])]
dy_tools      = [BoxZoomTool(), WheelZoomTool(dimensions=['width']), WheelZoomTool(dimensions=['height']), 
                 PanTool(), SaveTool(), BoxSelectTool(dimensions=['width'])]

gh_plot = figure(title = 'GH Data, m: [%i]' % count, 
                 plot_height = 300, 
                 plot_width = 950, 
                 tools = gh_tools, 
                 toolbar_location = "above")

gh_plot.line  ("x", "m", source = source, color = '#ff0000', line_width = 1)
gh_plot.circle("x", "m", source = source, color = '#ff0000', size = 2)

gh_plot.line  ("x", "c", source = source, color = '#0000ff', line_width = 1)
gh_plot.circle("x", "c", source = source, color = '#0000ff', size = 2)

anomaly_plot = figure(title = 'Is Anomaly: [%i]' % count, 
                      plot_height = 200, 
                      plot_width = 950, 
                      y_range = (-0.5,1.5), 
                      x_range = gh_plot.x_range, 
                      tools = anomaly_tools, 
                      toolbar_location = "above")

anomaly_plot.line  ("x", "a", source = source, color = '#000000', line_width = 1)
anomaly_plot.circle("x", "a", source = source, color = '#000000', size = 2)

dy_plot = figure(title = 'Derivative, mph: [%i]' % count, 
                 plot_height = 200, 
                 plot_width = 950, 
                 x_range = gh_plot.x_range,
                 tools = dy_tools, 
                 toolbar_location = "above")

dy_plot.line("x", "dy", source = source, color = '#ff0000', line_width = 1)

source.callback = CustomJS(args = dict(gh_plot = gh_plot), 
                           code = """var inds = cb_obj.get('selected')['1d'].indices;
                                     var d1 = cb_obj.get('data');
                                     console.log(d1)
                                     var kernel = IPython.notebook.kernel;
                                     IPython.notebook.kernel.execute("idx = " + inds);""")



set_button   = Button(description = "Set")
unset_button = Button(description = "Unset")
prev_button  = Button(description = 'Previous')
next_button  = Button(description = 'Next')
save_button  = Button(description = 'Save')

def on_set(b):
    global y_true
    y_true[i - k + np.array(idx)] = 1    
    source.data['a'] = y_true[i-k:i+n+k]
    push_notebook()
    
def on_unset(b):
    global y_true
    y_true[i - k + np.array(idx)] = 0    
    source.data['a'] = y_true[i-k:i+n+k]
    push_notebook()    
    
def on_next(b):
    global i, count
    i = i + n + k
    count = count + 1
    gh_plot.title.text = "GH Data: [%i]" % count
    anomaly_plot.title.text = "Is Anomaly: [%i]" % count
    source.data['m']  = Ym[i-k:i+n+k]
    source.data['c']  = Yc[i-k:i+n+k]
    source.data['dy'] = dY[i-k:i+n+k]
    source.data['a']  = y_true[i-k:i+n+k]
    push_notebook()
    
def on_prev(b):
    global i, count
    i = i - n - k 
    count = count - 1
    gh_plot.title.text = "GH Data: [%i]" % count
    anomaly_plot.title.text = "Is Anomaly: [%i]" % count
    source.data['m']  = Ym[i-k:i+n+k]
    source.data['c']  = Yc[i-k:i+n+k]
    source.data['dy'] = dY[i-k:i+n+k]
    source.data['a']  = y_true[i-k:i+n+k]
    push_notebook()
    
def on_save(b):
    json.dump(
        {
            'site_no': sid[si], 
            'is_anomaly': y_true.astype(np.uint8).tolist()
        }, 
        open('revision/%i.json' % sid[si], 'w')
    )   

set_button.on_click(on_set)
unset_button.on_click(on_unset)
next_button.on_click(on_next)
prev_button.on_click(on_prev)
save_button.on_click(on_save)

In [ ]:
display(HBox([set_button, unset_button, prev_button, next_button, save_button]))
show(column(gh_plot, dy_plot, anomaly_plot))